In [1]:
from pathlib import Path
import os

dataset_name = 'mercedes-benz-greener-manufacturing'

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path(f'../input/{dataset_name}')
else:
    import zipfile,kaggle
    path = Path(f'input/{dataset_name}')
    if not path.exists():
        kaggle.api.competition_download_cli(dataset_name)
        zipfile.ZipFile(f'{dataset_name}.zip').extractall(path)

In [2]:
import numpy as np
import pandas as pd

np.set_printoptions(linewidth=120)
pd.set_option('display.width', 120)

In [3]:
train = pd.read_csv(path/'train.csv.zip')
train

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,...,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,...,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,...,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y = train['y']
X = train.drop(['ID','y'], axis = 1)

In [5]:
num_cols = X.select_dtypes(include = 'number').columns.values
cat_cols = X.select_dtypes(include = 'object').columns.values

print(num_cols)
print('-' * 10)
print('cat_cols ', cat_cols)

['X10' 'X11' 'X12' 'X13' 'X14' 'X15' 'X16' 'X17' 'X18' 'X19' 'X20' 'X21' 'X22' 'X23' 'X24' 'X26' 'X27' 'X28' 'X29'
 'X30' 'X31' 'X32' 'X33' 'X34' 'X35' 'X36' 'X37' 'X38' 'X39' 'X40' 'X41' 'X42' 'X43' 'X44' 'X45' 'X46' 'X47' 'X48'
 'X49' 'X50' 'X51' 'X52' 'X53' 'X54' 'X55' 'X56' 'X57' 'X58' 'X59' 'X60' 'X61' 'X62' 'X63' 'X64' 'X65' 'X66' 'X67'
 'X68' 'X69' 'X70' 'X71' 'X73' 'X74' 'X75' 'X76' 'X77' 'X78' 'X79' 'X80' 'X81' 'X82' 'X83' 'X84' 'X85' 'X86' 'X87'
 'X88' 'X89' 'X90' 'X91' 'X92' 'X93' 'X94' 'X95' 'X96' 'X97' 'X98' 'X99' 'X100' 'X101' 'X102' 'X103' 'X104' 'X105'
 'X106' 'X107' 'X108' 'X109' 'X110' 'X111' 'X112' 'X113' 'X114' 'X115' 'X116' 'X117' 'X118' 'X119' 'X120' 'X122' 'X123'
 'X124' 'X125' 'X126' 'X127' 'X128' 'X129' 'X130' 'X131' 'X132' 'X133' 'X134' 'X135' 'X136' 'X137' 'X138' 'X139' 'X140'
 'X141' 'X142' 'X143' 'X144' 'X145' 'X146' 'X147' 'X148' 'X150' 'X151' 'X152' 'X153' 'X154' 'X155' 'X156' 'X157' 'X158'
 'X159' 'X160' 'X161' 'X162' 'X163' 'X164' 'X165' 'X166' 'X167' '

In [44]:
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression

In [45]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
transformer_steps = [
    ('ohe', OneHotEncoder(handle_unknown='infrequent_if_exist'), cat_cols),
]

transformer = ColumnTransformer(transformer_steps, remainder='passthrough')

pipeline = Pipeline([
    ('transform', transformer),
    ('scaler', StandardScaler()),
#     ('sel', SelectKBest(mutual_info_regression, k=50)),
    ('pca', PCA(n_components=200)),
    ('lr', LinearRegression())
])

In [80]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  array(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype=object))])),
                ('scaler', StandardScaler()), ('pca', PCA(n_components=200)),
                ('lr', LinearRegression())])

In [81]:
y_train_pred = pipeline.predict(X_train)
y_val_pred = pipeline.predict(X_val)

In [82]:
from sklearn.metrics import r2_score

print(r2_score(y_train, y_train_pred))
print(r2_score(y_val, y_val_pred))

0.5869324137463847
0.5839876929266387


## H20 AutoML

In [84]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.4.1" 2022-08-12; OpenJDK Runtime Environment Homebrew (build 17.0.4.1+0); OpenJDK 64-Bit Server VM Homebrew (build 17.0.4.1+0, mixed mode, sharing)
  Starting server from /Users/viktortnk/code/python/kaggle-mercedes-gm/.venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/4y/0z4t8csn3rvb02krzv5rmcp80000gn/T/tmp7lk4zroe
  JVM stdout: /var/folders/4y/0z4t8csn3rvb02krzv5rmcp80000gn/T/tmp7lk4zroe/h2o_viktortnk_started_from_python.out
  JVM stderr: /var/folders/4y/0z4t8csn3rvb02krzv5rmcp80000gn/T/tmp7lk4zroe/h2o_viktortnk_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_viktortnk_iy0i7t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [96]:
from h2o.sklearn import H2OAutoMLRegressor

pipeline = Pipeline([
    ('transform', transformer),
    ('regressor', H2OAutoMLRegressor(max_models=20, seed=42))
])

In [97]:
pipeline.fit(X_train, y_train.values)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
12:54:09.25: AutoML: XGBoost is not available; skipping it.
12:54:09.27: _train param, Dropping bad and constant columns: [C507, C524, C413, C446, C468, C467, C411, C466, C289, C475, C275, C471, C195]


12:54:09.666: _train param, Dropping bad and constant columns: [C507, C524, C413, C446, C468, C467, C411, C466, C289, C475, C275, C471, C195]

██
12:54:12.3: _train param, Dropping bad and constant columns: [C507, C524, C413, C446, C468, C467, C411, C466, C289, C475, C275, C471, C195]

██████
12:54:18.272: _train param, Dropping bad and constant columns: [C507, C524, C413, C446, C468, C467, C411, C466, C289, C475, C275, C471, C195]
12:54:20.154: _train param, Dropping bad and constant columns: [C507, C524, C413, C446, C468, C467, C411, C466, C289, C475, C275, C471, C195]

█
12:54:2

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  array(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype=object))])),
                ('regressor', H2OAutoMLRegressor(max_models=20, seed=42))])

In [99]:
y_train_pred = pipeline.predict(X_train)
y_val_pred = pipeline.predict(X_val)
print(r2_score(y_train, y_train_pred))
print(r2_score(y_val, y_val_pred))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
0.7259851929133172
0.5830943548325271
